# qBraid-SDK: Devices and Jobs

Enable [qBraid Quantum Jobs](https://docs.qbraid.com/en/latest/lab/quantumjobs.html) through the [qBraid CLI](https://docs.qbraid.com/en/latest/cli/qbraid.html):

In [1]:
!qbraid jobs enable qbraid_sdk

Successfully enabled qBraid Quantum Jobs in the qbraid_sdk environment.
Every AWS + IBM job you run will now be submitted through the qBraid API, so no access keys/tokens are necessary. 

To disable, run: `qbraid jobs disable qbraid_sdk`


You can check that the `jobs` keyword next to the qBraid SDK environment is now green.

In [2]:
!qbraid envs list

# installed environments:
#
qsharp                         /opt/.qbraid/environments/qsharp_b54crn
default                  jobs  /opt/.qbraid/environments/qbraid_000000
qiskit                   jobs  /home/jovyan/.qbraid/environments/qiskit_9y9siy
amazon_braket            jobs  /home/jovyan/.qbraid/environments/aws_braket_kwx6dl
intel                          /opt/.qbraid/environments/intel_zr7hfq
qbraid_sdk               jobs  /home/jovyan/.qbraid/environments/qbraid_sdk_9j9sjy



It's important to import the qBraid SDK only *after* you have enabled quantum jobs.

In [3]:
import qbraid

qbraid.__version__

'0.3.0'

## Unified quantum device search

List all of the 20+ QPUs and on-demand simulators available through qBraid Quantum Jobs.

In [4]:
from qbraid import get_devices

In [5]:
get_devices()

Provider,Name,qBraid ID,Status
AWS,DM1,aws_dm_sim,●
AWS,SV1,aws_sv_sim,●
AWS,TN1,aws_tn_sim,●
IBM,Belem,ibm_q_belem,●
IBM,Ext. stabilizer simulator,ibm_q_ext_stabilizer_sim,●
IBM,Jakarta,ibm_q_jakarta,●
IBM,Lagos,ibm_q_lagos,●
IBM,Lima,ibm_q_lima,●
IBM,MPS simulator,ibm_q_mps_sim,●
IBM,Manila,ibm_q_manila,○


Filter your device search with many different advanced query options! 

For example, let's find all gate-based QPUs that use at least 7 qubits:

In [6]:
get_devices(
    filters={
        "paradigm": "gate-based",
        "type": "QPU",
        "numberQubits": {"$gte": 7},
    }
)

For more query examples, see [Unified Device Search](https://docs.qbraid.com/en/latest/sdk/devices.html#unified-device-search) in the user guide.

## Create quantum circuit

Create a quantum circuit using any of one the qBraid support program types:

In [7]:
from qbraid import QPROGRAM_TYPES
from qbraid.interface import random_circuit

QPROGRAM_TYPES

['braket.circuits.circuit.Circuit',
 'cirq.circuits.circuit.Circuit',
 'qiskit.circuit.quantumcircuit.QuantumCircuit',
 'pyquil.quil.Program',
 'pytket._tket.circuit.Circuit']

In this example, we'll use a randomly generated Cirq circuit:

In [8]:
cirq_circuit = random_circuit("cirq", num_qubits=4, depth=4)

print(cirq_circuit)

      ┌──┐       ┌──┐
0: ─────@────@─────@────×───
        │    │     │    │
1: ────×┼────┼────Y┼────×───
       ││    │     │
2: ────┼@────@─────@────Y───
       │
3: ────×─────Y────Y─────────
      └──┘       └──┘


## Run circuit on any supported device

Copy the desired Device ID into the device wrapper function and you're on your way.

In [9]:
from qbraid import device_wrapper
from qbraid.devices.ibm import ibm_least_busy_qpu

In [10]:
aws_ionq_id = "aws_ionq"

qbraid_aws_device = device_wrapper("aws_ionq")
qbraid_aws_device.status

<DeviceStatus.ONLINE: 0>

In [11]:
ibm_qpu_id = ibm_least_busy_qpu()

qbraid_ibm_device = device_wrapper("ibm_q_belem")
qbraid_ibm_device.num_qubits

5

Initialize quantum jobs and collect results on any device through the same simple and consistent protocols. In this example, we'll compare results from running our bell circuit on the AWS and IBM state vector simulators:

In [12]:
get_devices(filters={"qbraid_id": {"$regex": "sv"}})

In [1]:
aws_sv_sim = device_wrapper("aws_sv_sim")
ibm_sv_sim = device_wrapper("ibm_q_sv_sim")

qbraid_aws_job = aws_sv_sim.run(cirq_circuit, shots=100)
qbraid_ibm_job = ibm_sv_sim.run(cirq_circuit, shots=100)

qbraid_ibm_job.status

## Monitor and manage your jobs from one location

In [14]:
from qbraid import get_jobs

In [15]:
get_jobs()

qBraid ID,Submitted,Status
aws_sv_sim-ryanjh88-qjob-f9d7kt1hw90q2dflr8nf,2023-04-20T22:45:25.026Z,COMPLETED
aws_sv_sim-ryanjh88-qjob-qjo8hvgybnuss37r74xo,2023-04-20T22:23:25.903Z,COMPLETED
ibm_q_sv_sim-ryanjh88-qjob-ffifddpi00bi3vgxp701,2023-04-20T22:23:23.729Z,COMPLETED
aws_dm_sim-ryanjh88-qjob-80fu1prgbxxbdcsaywmp,2023-04-20T22:23:09.779Z,COMPLETED
ibm_q_sv_sim-ryanjh88-qjob-lw2sv8ooy4t3lqrbxvkl,2023-04-20T22:22:55.692Z,COMPLETED
aws_sv_sim-ryanjh88-qjob-ljqolzju4qpdpkf07pkh,2023-04-20T22:22:44.859Z,COMPLETED
aws_sv_sim-ryanjh88-qjob-hzrur7urczi1plak6a3j,2023-04-20T22:22:43.591Z,COMPLETED
aws_sv_sim-ryanjh88-qjob-ne4eqj9jz0x2n83jaesn,2023-04-20T22:22:42.305Z,COMPLETED
aws_sv_sim-ryanjh88-qjob-p2nt5cve64sfzsc4ag2z,2023-04-20T22:22:40.750Z,COMPLETED
ibm_q_qasm_sim-ryanjh88-qjob-l213l2854ibxh3o80a5q,2023-04-20T22:22:39.026Z,COMPLETED


In [ ]:
aws_result = qbraid_aws_job.result()
ibm_result = qbraid_ibm_job.result()

## Easily compare results

In [ ]:
aws_counts = aws_result.measurement_counts()
ibm_counts = ibm_result.measurement_counts()

print(aws_sv_sim.name)
print(f"counts: {aws_counts}\n")

print(ibm_sv_sim.name)
print(f"counts: {ibm_counts}\n")

In [ ]:
aws_result.plot_counts()

In [ ]:
ibm_result.plot_counts()